# 서약
* 당연한 것을 아래와 같이 한번 다지고 가도록 하겠습니다. 아래 보고서는 **본인의 힘만으로 작성**해야하며, 다른 수강생에게 직접적인 질문, 복사 하는 행위는 모두 금지합니다 
  * 예를 들어서, 본 프로젝트의 코드 셀을 완성하는데 직접적인 질문 또는 복사하는 경우는 모두 금지합니다
  * 수업에서 제공한 코드, 노트북은 모두 재활용가능하며, 카피로 규정하지 않습니다
  * 수업 자료 이외에 참고자료가 있다면, 출처와 사용 부분에 모두 표시하는 경우는 모두 합당한 자료로 인정하겠습니다
  
* 위에 대해서 모두 이해하고 동의했다면, 아래 `서약글`에 다음을 작성해주세요:

"본인은 위 서약글을 이해하고 동의하며, 프로젝트를 수행하는데 있어서 반칙을 할 경우 (제공자 포함) 본 프로젝트에 대한 점수가 반영되지 않는다는 것에 동의 합니다."

**서약서**

이름: 박성욱

학번: 20155319

서약글:  본인은 위 서약글을 이해하고 동의하며, 프로젝트를 수행하는데 있어서 반칙을 할 경우(제공자 포함)본 프로젝트에 대한 점수가 반영되지 않는다는 것에 동의합니다.

# Final Project 

## Project: 새 품종 분류하기

## Due date: 2021.06.15

---

* 아래 여러 셀에서 코드를 완성하는 부분을 수행하고, 설명을 요구하는 부분은 보고서에 설명을 최대한 자세하게 적어주세요. 기준은 본인이 이해하고 있다는 것을 표현할 수 있는 부분을 모두 적으시면 됩니다.
  * 답을 작성하는 원칙은 **보고서**를 작성한다고 생각하시면 됩니다
  * 내가 알고 있는 부분을 충실하게 **글로 표현** 하는 것 또한 중요한 연습입니다 
  * 코드 작성은 **주석**으로 설명하시기 바랍니다
  
  
> **제출방법**: 
* 보고서에는 코드 캡쳐 첨부이외에도, 각 코드를 작성하는 방법론과 설명을 작성해야하는 **서술형 문제**도 포함되어 있습니다.
* 서술형 문제는 채점하는 중요한 기준이 됩니다. 성실하게 작성해주세요.
* 서술형 문제에 대한 답변은 첨부된 보고서에 작성하면 됩니다. 


### 목표: 앱을 위한 인공지능 알고리즘 개발 ^ㅡ^
* 본 보고서에서는 모바일/웹앱을 위한 인공지능 알고리즘을 개발하라는 업무를 부여받았다고 가정합니다
* 프로젝트가 완성본은, 사용자가 제공하는 image를 받아서 새의 종(種)을 예측합니다


---

## Step 0: Import library

- 필요하다 생각되는 라이브러리를 미리 import해 놓은 항목입니다.(참고)
- 필요 없는 라이브러리를 제거하거나 필요한 라이브러리를 추가하셔도 됩니다.

In [ ]:
import shutil
import os
import glob
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import pandas as pd
import cv2
from pathlib import Path
from collections import OrderedDict

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
import torch.nn.init

from sklearn.model_selection import KFold

from tqdm import tqdm

## Step 1: Dataset 준비하기

In [ ]:
#코드작성
#data의 경로를 저장한 변수
path = 'C:/Jupyter Notebook/DL_homework/final_project/detect-the-bird/imgs/'

#glob을 통하여 train image, test image를 불러올 경로를 각각 train_files, test_path의 리스트안에 저장
# train_files = glob.glob(path + 'train/*/*.*')#####
test_path = glob.glob(path + 'test/*')

#train과 test data가 손실없이 정확히 불러와졌는지 check
# print('train_data : ', len(train_files))#####
print('test_data : ', len(test_path))

#cuda
cuda_ = torch.cuda.is_available()

#cuda를 사용할 수 있으면 device는 cuda로 아니면 cpu사용
if cuda_:
    device = 'cuda'
    print('device : ', device)
    
else:
    device = 'cpu'
    print('device : ', device)

In [ ]:
#input data가 얼만큼 들어갈지 설정할 batch_size 설정
batch_size = 20
#이미지의 크기를 고정해주기 위한 image_size 설정
image_size = 256

#train data를 불러올 path 설정
train_path = path + 'train'

#train data의 mean, std를 구하기 위한 transform
transform_train = transforms.Compose([transforms.Resize((image_size,image_size)),
                                      transforms.CenterCrop(224),
                                     transforms.ToTensor()])

In [ ]:
#Imagefolder를 통해 각각의 폴더별로 클래스를 생성하고 그에 맞는 label을 할당
train_folder = datasets.ImageFolder(train_path, transform = transform_train)
#input으로 집어넣을 수 있도록 loader구성
train_loader = torch.utils.data.DataLoader(train_folder, batch_size = batch_size, num_workers = 4, shuffle = True)

#예측을 할 때 사용할 classes미리 선언
classes = train_folder.classes

#classes가 빠짐없이 잘 되어있는지 확인
print('num_class : ', len(classes))
print('classes : ', classes)

In [ ]:
#이미지 data에 정규화를 시켜줄 때 각각 RGB의 평균과 표준편차로 transform하기 때문에 각각 RGB의 평균을 내줌
meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in train_loader]
stdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in train_loader]

meanR = np.mean([m[0] for m in meanRGB])
meanG = np.mean([m[1] for m in meanRGB])
meanB = np.mean([m[2] for m in meanRGB])

stdR = np.mean([s[0] for s in stdRGB])
stdG = np.mean([s[1] for s in stdRGB])
stdB = np.mean([s[2] for s in stdRGB])

print(meanR, meanG, meanB)
print(stdR, stdG, stdB)

In [ ]:
# #validation size
# valid_size = 0.3

# #train data의 total 개수 저장
# num_train = len(train_folder)

# #train data의 개수만큼 idx_list라는 변수에 리스트로 설정 train은 총 2468개 이므로 0~2467로 이루어진 리스트 형성
# idx_list = list(range(num_train))

# #리스트를 섞어줌
# np.random.shuffle(idx_list)

# #위에 정해준 validation 크기만큼 train과 validation을 위한 idx split
# split = int(np.floor(valid_size*num_train))
# train_idx, valid_idx = idx_list[split:], idx_list[:split]

# #위에서 나눈 train idx와 valid idx로 각각 train data와 validation data를 할당
# train_sampler = SubsetRandomSampler(train_idx)
# valid_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
# #data가 잘 나누어졌는지 확인
# print('train data : ' ,len(train_sampler))
# print('valid data : ', len(valid_sampler))

In [ ]:
#train data에 적용할 transform
transform_train = transforms.Compose([transforms.Resize((image_size,image_size)), #image의 size를 위에서 정한 image size로 resize
                                      transforms.CenterCrop(224), #resize한 이미지를 224x224로 잘라서 input에 넣는다. 영상의 가장자리보다 중심에 더 중요한
                                                                  #정보가 있을 것으로 예상하여서
                                      transforms.RandomHorizontalFlip(p=0.5), #0.5의 확률로 뒤집기
                                      transforms.RandomRotation(10), #이미지를 10도 돌린다.
                                      transforms.RandomGrayscale(p=0.1),#0.1의 확률로 흑백 이미지로 변환
                                     transforms.ToTensor(), #데이터 타입을
                                     transforms.Normalize([meanR, meanG, meanB],
                                                            [stdR, stdG, stdB])])

transform_valid = transforms.Compose([transforms.Resize((image_size, image_size)),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                     transforms.Normalize([meanR, meanG, meanB],
                                                            [stdR, stdG, stdB])])

## Step 2: Dataset 에 대한 Data Loaders 구성

In [ ]:
# #코드작성
train_folder = datasets.ImageFolder(train_path, transform = transform_train)
# valid_folder = datasets.ImageFolder(train_path, transform = transform_valid)

# train_loader = torch.utils.data.DataLoader(train_folder, batch_size = batch_size, num_workers = 4, sampler = train_sampler)
# valid_loader = torch.utils.data.DataLoader(valid_folder, batch_size = batch_size, num_workers = 4, sampler = valid_sampler)

---

## Step 3: Neural Network 생성
- Pretrained model을 허용하지 않습니다. (직접 모델을 설계해 주세요)


In [ ]:
class model6(nn.Module):
    def __init__(self):
        super().__init__()
        
        
        
        self.layer1 = nn.Sequential(nn.Conv2d(3,64,7,1,padding = 3), #224
                            nn.BatchNorm2d(64),
                            nn.ReLU(True),
                            nn.MaxPool2d(2,2)) #112

        self.layer2 = nn.Sequential(nn.Conv2d(64,128,1,1,padding = 0), #112
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(True),
                                    nn.Conv2d(128,256,3,1,padding = 1), #112
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2),#56
                                    nn.Conv2d(256,128,1,1,padding = 0),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(True))

        self.layer3 = nn.Sequential(nn.Conv2d(128,256,1,1,padding = 0),
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(True),
                                    nn.Conv2d(256,512,3,1,padding = 1), 
                                    nn.BatchNorm2d(512),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2),#28
                                    nn.Conv2d(512,256,1,1,padding = 0),
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(True))

        self.layer4 = nn.Sequential(nn.Conv2d(256,512,1,1,padding = 0),
                                    nn.BatchNorm2d(512),
                                    nn.ReLU(True),
                                    nn.Conv2d(512,1024,3,1,padding = 1), 
                                    nn.BatchNorm2d(1024),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2),#14
                                    nn.Conv2d(1024,512,1,1,padding = 0),
                                    nn.BatchNorm2d(512),
                                    nn.ReLU(True))

        self.layer5 = nn.Sequential(nn.Conv2d(512,1024,1,1,padding = 0),
                                    nn.BatchNorm2d(1024),
                                    nn.ReLU(True),
                                    nn.Conv2d(1024,2048,3,1,padding = 1),
                                    nn.BatchNorm2d(2048),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2),#7
                                    nn.Conv2d(2048,1024,1,1,padding = 0),
                                    nn.BatchNorm2d(1024),
                                    nn.ReLU(True))

        self.fc = nn.Sequential(nn.Dropout(0.8),
                                nn.Linear(1024*7*7, 1000),
                                nn.BatchNorm1d(1000),
                                nn.ReLU(True),
                                nn.Dropout(0.8),
                                nn.Linear(1000, 500),
                                nn.BatchNorm1d(500),
                                nn.ReLU(True),
                                nn.Dropout(0.8),
                                nn.Linear(500, 250),
                                nn.BatchNorm1d(250),
                                nn.ReLU(True),
                                nn.Linear(250, len(classes))
                               )

        self.dropout = nn.Dropout(0.5)
        
    #모델의 forward연산
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        #flatten
        x = x.view(x.shape[0], -1)
        
        x = self.dropout(x)
        
        x = self.fc(x)
        
        return x

# model 생성
model = model6()
#GPU연산
model.to(device)

---

 ## Step 4: Cost (Loss) Function 과 Optimizer 선택
 
 [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) 및 [optimizer](http://pytorch.org/docs/stable/optim.html)를 선택하여 코드를 완성하세요.
 
 위 링크에서 다양한 Loss Function과 Optimize Function을 확인 할 수 있습니다
 

In [ ]:
#코드작성
#crossentropy loss 사용
criterion = nn.CrossEntropyLoss()

#adam optimizer사용
optimizer = optim.Adam(model.parameters(), lr = 1e-2)

#원하는 epoch에 사용할 scheduler 사용 주로 원하는 epoch에 lr을 감소시킬 수 있는 scheduler를 사용
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80], gamma=0.1)
# scheduler_down = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 20, eta_min = 1e-3)
# scheduler_up = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=10)

---

## Step 5: 구성한 모델에 대한 Train and Validate 진행

* 코드 전체를 주석으로 설명하세요
* Epoch 별로 Loss나 Accuracy를 출력하여 학습 진행 과정을 확인 할 수 있도록 합니다
* 출력 예시는 주어지나 정해진 형식은 없습니다
* 최적의 모델 저장

예제:
```
Started Training...
Epoch: 1 	Training Loss: 3.317162 	Validation Loss: 4.162958
Epoch: 2 	Training Loss: 2.420140 	Validation Loss: 4.182362
...
...
Finished training
```

In [ ]:
#model의 weight가 적용될 변수 선언
pt_file = 'model weight'

#몇 번의 epoch을 돌것인가
n_epochs = 200

#만약 20번 동안 weight save가 되지 않으면 해당 학습을 강제로 종료
n_epochs_stop = 20
early_stop = False

#train data를 5분할 하여 각각 한 번씩 validation set으로 사용
kf = KFold(n_splits=5, shuffle=True)

#각각 train index와 validation index를 받아옴
for fold, (train_ind, valid_ind) in enumerate(kf.split(train_folder)):
    
    #index를 통해 train data와 validation data를 나눠줌
    train_sampler_kfold = SubsetRandomSampler(train_ind)
    valid_sampler_kfold = SubsetRandomSampler(valid_ind)
    
    #전체 data 수, train data, validation data의 수를 확인
    print(f'Total data : {len(train_folder)} Train data : {len(train_sampler_kfold)} Valid data : {len(valid_sampler_kfold)}')    
    
    #각각의 transform 적용을 위한 코드
    train_folder_kfold = datasets.ImageFolder(train_path, transform = transform_train)
    valid_folder_kfold = datasets.ImageFolder(train_path, transform = transform_valid)
    
    #train loader
    train_loader_kfold = torch.utils.data.DataLoader(train_folder, batch_size=batch_size,
    sampler=train_sampler_kfold, num_workers=4)
    
    #validation loader
    valid_loader_kfold = torch.utils.data.DataLoader(valid_folder, batch_size=batch_size, 
    sampler=valid_sampler_kfold, num_workers=4)
    
#     valid_loader_kfold = transform_valid(valid_loader_kfold)
    
    #각 fold별로 새로 학습하기 위해 model을 선언
    model = model6()
    #gpu연산을 위함.
    model.to(device)
    
    #loss function
    criterion = nn.CrossEntropyLoss()
    #Adam optimizer
    optimizer = optim.Adam(model.parameters(), lr = 2e-2)
    #scheduler
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80], gamma=0.1)
    
    #earlystop을 사용하고 싶으면 해당 부분에서 True로 아니면 False
    earlystop = True
    
    #scheduler를 사용하고 싶으면 해당 부분에서 True로 아니면 False
    scheduler_ = True
    
    #weight save가 얼마나 안됐는지 체크를 하여 early stop
    epochs_no_improve = 0
    
    #학습괴정에서의 train, validation loss를 저장
    train_loss = torch.zeros(n_epochs)
    valid_loss = torch.zeros(n_epochs)

    #학습 과정에서의 train, validation accuracy를 저장
    train_acc = torch.zeros(n_epochs)
    valid_acc = torch.zeros(n_epochs)
    
    #validation loss를 통해 이전 epoch과 비교하여 줄어들었으면 weight를 저장하기 위한 변수
    valid_loss_min = np.Inf
    
    #선언한 epoch만큼 for문을 반복
    for e in range(0, n_epochs):   
        ###################
        # train the model #
        ###################

        #현재 epoch의 learning rate를 확인하기 위해서 scheduler 적용시 잘 적용이 되었나 확인하기 위함이다.
        for param_group in optimizer.param_groups:
                train_lr = param_group['lr']

        #train mode로 전환
        model.train()

        #train loader를 통해 input data와 labels를 batch size만큼 떼어옴
        for data, labels in tqdm(train_loader_kfold): 
            
            #받아온 data와 labels를 gpu연산을 하기 위한 코드
            data, labels = data.to(device), labels.to(device)
            
            #학습 과정에서 미분을 통해 얻은 기울기를 0으로 초기화
            optimizer.zero_grad()
            
            #data를 model에 input으로 넣어 output 출력
            logits = model(data)
            
            logits = logits.squeeze()
            
            #예측값과 label값을 비교하여 loss를 계산
            loss = criterion(logits, labels)
            # backward pass: compute gradient of the loss with respect to model parameters
            
            #loss를 통해 기울기를 계산한다.
            loss.backward() #Expected input batch_size (49) to match target batch_size (64)오류 발생
            #계산한 기울기를 통해 learning rate만큼 weight를 업데이트
            optimizer.step()

            #train loss를 업데이트
            train_loss[e] += loss.item()

            #softmax를 통해 확률값을 구한다.
            ps = F.softmax(logits, dim=1)
            #구한 확률값중 가장 큰 값을 top_class에 적용
            top_p, top_class = ps.topk(1, dim=1)
            
            #true label과 predict label을 비교하여 정확도를 계산
            equals = top_class == labels.reshape(top_class.shape)
            train_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu()

        #train loss와 train accuracy를 계산
        train_loss[e] /= len(train_loader_kfold)
        train_acc[e] /= len(train_loader_kfold)


        ######################    
        # validate the model #
        ######################
        
        #autograd를 종료
        with torch.no_grad():
            #evaluation mode로 전환
            model.eval()
            #validation dataset을 받아옴
            for data, labels in tqdm(valid_loader_kfold):
                #data와 label을 gpu연산을 할 수 있도록
                data, labels = data.to(device), labels.to(device)
                #valiation set을 input으로 넣어 out
                logits = model(data)
                #loss함수를 통해 loss를 계산
                loss = criterion(logits, labels)
                #validation loss를 추가해줌
                valid_loss[e] += loss.item()
                #모델의 output을 확률값으로 변경해준다.
                ps = F.softmax(logits, dim=1)
                #구한 확률값중 가장 큰 값을 top_class에 적용한다.
                top_p, top_class = ps.topk(1, dim=1)
                #true label과 predict label을 비교하여 정확도를 계산
                equals = top_class == labels.reshape(top_class.shape)
                valid_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu()

        #validation loss와 accuracy를 계산
        valid_loss[e] /= len(valid_loader_kfold)
        valid_acc[e] /= len(valid_loader_kfold)

        #scheduler batchsize때문에 원하는 epoch에 lr이 줄어들지 않아 해당 위치에 배치
        if scheduler_:
            scheduler.step()

        #현재 학습의 폴드와 기본 세팅값을 표기해줌
        print(f'Fold : {fold} Learning rate : {train_lr:.6f}  no_improve_epoch : {epochs_no_improve}  Early Stop : {earlystop}   Scheduler : {scheduler_}')

        # train 상태를 출력해줌
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            e+1, train_loss[e], valid_loss[e]))

        #validation 상태를 출력해줌 
        print('Epoch: {} \tTraining accuracy: {:.6f}% \tValidation accuracy: {:.6f}%'.format(
            e+1, train_acc[e]*100, valid_acc[e]*100))

        #이전의 validation loss와 비교하여 크거나 같으면 model weight를 저장
        if valid_loss[e] <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss[e]))
            torch.save(model.state_dict(), f'{pt_file}_{fold}.pt')
            valid_loss_min = valid_loss[e]

            #early stop 초기화
            epochs_no_improve = 0
        else:
            #weight save가 되지 않으면 early stop count를 하나씩 추가
            epochs_no_improve += 1

        #early stop
        if earlystop:
            if e > 5 and epochs_no_improve == n_epochs_stop:
                print('Early stopping!')
                early_stop = True
                break
            else:
                continue

---

## Step 6: CNN model training/validation 분석
   * tranining loss와 validation loss 그래프를 통해서 분석


In [ ]:
#mtrain, validation loss plot
plt.plot(train_loss, label = 'Training loss')
plt.plot(valid_loss, label = 'validation loss')
plt.legend()
plt.savefig(f'result_.png')

* 최적의 모델 로드

In [ ]:
#fold n번째 weight를 불러오기 위한 모델들 생성
fold1 = model6()

#fold1 weight load
fold1.load_state_dict(torch.load('model weight_0.pt'))
#gpu연산
fold1.to(device)
#evaluation mode로 전환
fold1.eval()

In [ ]:
fold2 = model6()

fold2.load_state_dict(torch.load('model weight_1.pt'))
fold2.to(device)
fold2.eval()

In [ ]:
fold3 = model6()

fold3.load_state_dict(torch.load('model weight_2.pt'))
fold3.to(device)
fold3.eval()

In [ ]:
fold4 = model6()

fold4.load_state_dict(torch.load('model weight_3.pt'))
fold4.to(device)
fold4.eval()

In [ ]:
fold5 = model6()

fold5.load_state_dict(torch.load('model weight_4.pt'))
fold5.to(device)
fold5.eval()

---

## Step 7: Predict with Test Data 


### 예시 코드를 제공해 드립니다. 필요한 부분을 채워 사용하시거나 직접 코드를 작성 하셔도 됩니다

In [ ]:
class test_dataset(Dataset):
    def __init__(self,imgpath,transform=None):
        #img path와 transform의 상태를 받아옴
        self.imgpath = imgpath
        self.transform = transform
        #test img의 개수
    def __len__(self):
        return len(self.imgpath)
        #데이터를 받아옴
    def __getitem__(self,idx):
        #transform적용
        x = self.transform(Image.open(self.imgpath[idx]).convert('RGB'))
        #gpu연산
        x = x.to(device)
        
        return x

In [ ]:
transform_test = transform_valid

### test dataset과 dataloader 생성


In [ ]:
test_data = test_dataset(test_path, transform_test)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 1, num_workers = 0, shuffle = False)

In [ ]:
classes

### Load Ensemble Model

In [ ]:
class e1(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(3,64,7,1,padding = 3), #224
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2)) #112
        
        self.layer2 = nn.Sequential(nn.Conv2d(64,64,1,1,padding = 0), #112
                                    nn.BatchNorm2d(64),
                                    nn.Conv2d(64,64,3,1,padding = 1), #112
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2), #56
                                    nn.Conv2d(64,256, 1,1,padding = 0), 
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2)) #28
        
        self.layer3 = nn.Sequential(nn.Conv2d(256, 256, 1, 1, padding = 0), #28
                                    nn.BatchNorm2d(256),
                                    nn.Conv2d(256, 512, 3,1,padding =1), #28
                                    nn.BatchNorm2d(512),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2), #14
                                    nn.Conv2d(512, 1024, 3, 1, padding =1),
                                    nn.BatchNorm2d(1024),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2)) #7
        
        self.layer4 = nn.Sequential(nn.Conv2d(1024, 1024, 1, 1, padding = 0),
                                    nn.BatchNorm2d(256),
                                    nn.Conv2d(1024, 2048, 3, 1, padding = 1),
                                    nn.BatchNorm2d(2048),
                                    nn.ReLU(True),
                                    nn.Conv2d(2048, 1024, 1, 1, padding = 0),
                                    nn.BatchNorm2d(1024),
                                    nn.ReLU(True))

        


        self.fc = nn.Sequential(nn.Dropout(0.8),
                                nn.Linear(1024*7*7, 1000),
                                nn.BatchNorm1d(1000),
                                nn.ReLU(True),
                                nn.Dropout(0.8),
                                nn.Linear(1000, 500),
                                nn.BatchNorm1d(500),
                                nn.ReLU(True),
                                nn.Dropout(0.8),
                                nn.Linear(500, 250),
                                nn.BatchNorm1d(250),
                                nn.ReLU(True),
                                nn.Linear(250, len(classes))
                                )
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.5)
        
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = x.view(x.shape[0], -1)
        
        x = self.dropout(x)
        
        x = self.fc(x)
        
        return x
    
class e2(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layer1 = nn.Sequential(nn.Conv2d(3,64,7,1,padding = 3), #224
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2)) #112
        
        self.layer2 = nn.Sequential(nn.Conv2d(64,64,1,1,padding = 0), #112
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(True),
                                    nn.Conv2d(64,64,3,1,padding = 1), #112
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2), #56
                                    nn.Conv2d(64,256, 1,1,padding = 0), 
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2)) #28
        
        self.layer3 = nn.Sequential(nn.Conv2d(256, 256, 1, 1, padding = 0), #28
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(True),
                                    nn.Conv2d(256, 512, 3,1,padding =1), #28
                                    nn.BatchNorm2d(512),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2), #14
                                    nn.Conv2d(512, 1024, 3, 1, padding =1),
                                    nn.BatchNorm2d(1024),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2)) #7
        
        self.layer4 = nn.Sequential(nn.Conv2d(1024, 1024, 1, 1, padding = 0),
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(True),
                                    nn.Conv2d(1024, 2048, 3, 1, padding = 1),
                                    nn.BatchNorm2d(2048),
                                    nn.ReLU(True),
                                    nn.Conv2d(2048, 1024, 1, 1, padding = 0),
                                    nn.BatchNorm2d(1024),
                                    nn.ReLU(True))

        


        self.fc = nn.Sequential(nn.Dropout(0.9),
                                nn.Linear(1024*7*7, 1000),
                                nn.BatchNorm1d(1000),
                                nn.ReLU(True),
                                nn.Dropout(0.9),
                                nn.Linear(1000, 500),
                                nn.BatchNorm1d(500),
                                nn.ReLU(True),
                                nn.Dropout(0.9),
                                nn.Linear(500, 250),
                                nn.BatchNorm1d(250),
                                nn.ReLU(True),
                                nn.Linear(250, len(classes))
                                )
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.5)
        
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        
        x = x.view(x.shape[0], -1)
        
        x = self.dropout(x)
        
        x = self.fc(x)
        
        return x

class e3(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layer1 = nn.Sequential(nn.Conv2d(3,64,7,1,padding = 3), #224
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2)) #112
        
        self.layer2 = nn.Sequential(nn.Conv2d(64,64,1,1,padding = 0), #112
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(True),
                                    nn.Conv2d(64,64,3,1,padding = 1), #112
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2), #56
                                    nn.Conv2d(64,256, 1,1,padding = 0), 
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2)) #28
        
        self.layer3 = nn.Sequential(nn.Conv2d(256, 256, 1, 1, padding = 0), #28
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(True),
                                    nn.Conv2d(256, 512, 3,1,padding =1), #28
                                    nn.BatchNorm2d(512),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2), #14
                                    nn.Conv2d(512, 1024, 3, 1, padding =1),
                                    nn.BatchNorm2d(1024),
                                    nn.ReLU(True),
                                    nn.MaxPool2d(2,2)) #7
        
        self.layer4 = nn.Sequential(nn.Conv2d(1024, 1024, 1, 1, padding = 0),
                                    nn.BatchNorm2d(1024),
                                    nn.ReLU(True),
                                    nn.Conv2d(1024, 2048, 3, 1, padding = 1),
                                    nn.BatchNorm2d(2048),
                                    nn.ReLU(True),
                                    nn.Conv2d(2048, 1024, 1, 1, padding = 0),
                                    nn.BatchNorm2d(1024),
                                    nn.ReLU(True))


        self.fc = nn.Sequential(nn.Dropout(0.8),
                                nn.Linear(1024*7*7, 1000),
                                nn.BatchNorm1d(1000),
                                nn.ReLU(True),
                                nn.Dropout(0.8),
                                nn.Linear(1000, 500),
                                nn.BatchNorm1d(500),
                                nn.ReLU(True),
                                nn.Dropout(0.8),
                                nn.Linear(500, 250),
                                nn.BatchNorm1d(250),
                                nn.ReLU(True),
                                nn.Linear(250, len(classes))
                                )
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.5)
        
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = x.view(x.shape[0], -1)
        
        x = self.dropout(x)
        
        x = self.fc(x)
        
        return x

In [ ]:
#ensemble
em1 = e1()

em1.load_state_dict(torch.load('model4_77.pt'))
em1.to(device)
em1.eval()

In [ ]:
em2 = e2()
em2.load_state_dict(torch.load('model_5_80.pt'))
em2.to(device)

em2.eval()

In [ ]:
em3 = e3()
em3.load_state_dict(torch.load('model6_83_1.pt'))
em3.to(device)

em3.eval()

In [ ]:
em4 = e2()
em4.load_state_dict(torch.load('model5_71.pt'))
em4.to(device)

em4.eval()

In [ ]:
em5 = e3()
em5.load_state_dict(torch.load('model6_81_45.pt'))
em5.to(device)

em5.eval()

In [ ]:
pred=[]
#test loader를 통해 test image를 받아온다.
for data in test_loader:
    #test image를 input에 적용
    logits1 = fold1(data)
    logits2 = fold2(data)
    logits3 = fold3(data)
    logits4 = fold4(data)
    logits5 = fold5(data)
    logits6 = em1(data)
    logits7 = em2(data)
    logits8 = em3(data)
    logits9 = em4(data)
    logits10 = em5(data)

    #model을 통과한 클래스들의 확률값을 구해준다.
    ps1 = F.softmax(logits1, dim=1)
    ps2 = F.softmax(logits2, dim=1)
    ps3 = F.softmax(logits3, dim=1)
    ps4 = F.softmax(logits4, dim=1)
    ps5 = F.softmax(logits5, dim=1)
    ps6 = F.softmax(logits6, dim=1)
    ps7 = F.softmax(logits7, dim=1)
    ps8 = F.softmax(logits8, dim=1)
    ps9 = F.softmax(logits9, dim=1)
    ps10 = F.softmax(logits10, dim=1)

    #모든 확률값의 평균을 적용하여 확률을 앙상블
    ps = (ps1+ps2+ps3+ps4+ps5+ps6+ps7+ps8+ps9+ps10)/10
    
     #확률값중 가장 높은 확률의 class를 return
    _, top_class = ps.topk(1, dim =1)
    
    #class를 정수형으로 바꿔 0~14의 값으로 변환해준다.
    pred += [int(top_class)]

### Predict 수행

예측 결과 인덱스를 저장한 pred 값을 사용해 클래스를 매칭하기 위한 class_name을 담아 놓은 리스트입니다.

주의할 점은 Train과정에서 사용된 class lable값과 같은 순서로 저장이 되어 있어야 한다는 점입니다


In [ ]:
id=[]
category =[]

for i in range(len(test_path)):
    id.append(test_path[i].split('/')[-1].split('\\')[-1])
    category.append(classes[pred[i]])
pd.DataFrame({'Id':id,'Category':category}).to_csv('submission1.csv',index=False)

## ---